Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [11]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(45),
    transforms.ToTensor(),
    transforms.Resize((128,128)),
])
train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

classes = train_data.classes
num_classes = len(train_data.classes)

In [ ]:
num_input_channels = 1
num_output_channels = 16
image_size = train_data[0][0].shape[1]

class MultiClassImageClassifier(nn.Module):
    def __init__(self, num_classes):
        super(MultiClassImageClassifier, self).__init__()
        self.conv1 = nn.Conv2d(num_input_channels, num_output_channels, kernel_size = 3, stride = 1, padding = 1)
        self.elu = nn.ELU()
        self.maxpool = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.flatten = nn.Flatten()

        self.fc = nn.Linear(num_output_channels * (image_size//2)**2, num_classes)


    def forward(self, x):
        x = self.conv1(x)
        x = self.elu(x)
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.fc(x)

        return x

In [13]:
dataloader_train = DataLoader(train_data, batch_size=10, shuffle=True)


def train_model(optimizer, net, num_epoch):
    num_processed = 0
    criterion = nn.CrossEntropyLoss()
    for epoch in range(num_epoch):
        running_loss = 0
        num_processed = 0
        for feauters, labels in dataloader_train:
            optimizer.zero_grad()
            outputs = net(feauters)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            num_processed += len(labels)
        print(f'epoch {epoch}, loss: {running_loss / num_processed}')
    train_loss = running_loss / len(dataloader_train)

net = MultiClassImageClassifier(num_classes) 
optimizer = optim.Adam(net.parameters(), lr=0.001)

train_model(
    optimizer = optimizer, 
    net = net,
    num_epoch=1,
)

epoch 0, loss: 0.04269253253931335


In [14]:
dataloader_test = DataLoader(test_data, batch_size=10, shuffle=False)

accuracy_metric = Accuracy(task='multiclass', num_classes=num_classes)
precision_metric = Precision(task='multiclass', num_classes=num_classes, average=None)
recall_metric = Recall(task='multiclass', num_classes=num_classes, average=None)
net.eval()
predictions = []
for i, (features, labels) in enumerate(dataloader_test):
    output = net.forward(features.reshape(-1, 1, image_size, image_size))
    cat = torch.argmax(output, dim=-1)
    predictions.extend(cat.tolist())
    accuracy_metric(cat, labels)
    precision_metric(cat, labels)
    recall_metric(cat, labels)

# Compute the metrics
accuracy = accuracy_metric.compute().item()
precision = precision_metric.compute().tolist()
recall = recall_metric.compute().tolist()
print('Accuracy:', accuracy)
print('Precision (per class):', precision)
print('Recall (per class):', recall)

Accuracy: 0.8737000226974487
Precision (per class): [0.778268575668335, 0.9887179732322693, 0.8238396644592285, 0.8969072103500366, 0.7512864470481873, 0.966292142868042, 0.7071078419685364, 0.9537329077720642, 0.9686868786811829, 0.909599244594574]
Recall (per class): [0.8809999823570251, 0.9639999866485596, 0.781000018119812, 0.8700000047683716, 0.8759999871253967, 0.9459999799728394, 0.5770000219345093, 0.9070000052452087, 0.9589999914169312, 0.9760000109672546]
